# function calling(tools)

https://python.langchain.com/v0.2/docs/how_to/function_calling/


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [3]:
llm.invoke("How will the weather be in munich today?")

AIMessage(content="I'm sorry, I am not able to provide real-time weather updates. I recommend checking a reliable weather website or app for the most up-to-date information on the weather in Munich today.", response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 17, 'total_tokens': 55}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-913329a3-3c65-4a40-a843-271beeaeadfd-0', usage_metadata={'input_tokens': 17, 'output_tokens': 38, 'total_tokens': 55})

In [4]:
from langchain_core.tools import tool

@tool
def fake_weather_api(city: str) -> str:
    """Check the weather in a specified city."""
    return "Sunny, 22°C"

@tool
def outdoor_seating_availability(city: str) -> str:
    """Check if outdoor seating is available at a specified restaurant in a given city."""
    return "Outdoor seating is available."


tools = [fake_weather_api, outdoor_seating_availability]

In [ ]:
# from langchain_core.pydantic_v1 import BaseModel, Field

# class WeatherCheck(BaseModel):
#     """Check the weather in a specified city."""

#     city: str = Field(..., description="Name of the city to check the weather for")


# class OutdoorSeatingCheck(BaseModel):
#     """Check if outdoor seating is available at a specified restaurant in a given city."""

#     city: str = Field(..., description="Name of the city where the restaurant is located")


# tools = [WeatherCheck, OutdoorSeatingCheck]

In [5]:
llm_with_tools = llm.bind_tools(tools)

In [6]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FDxn6c5ARe6q12LlgehasQT3', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 90, 'total_tokens': 107}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e9abe222-d625-41f2-b117-c460e024c325-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_FDxn6c5ARe6q12LlgehasQT3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 90, 'output_tokens': 17, 'total_tokens': 107})

In [7]:
result = llm_with_tools.invoke("How will the weather be in munich today? I would like to eat outside if possible")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qcyCL7l0m8TSeAViDuLe4Ci2', 'function': {'arguments': '{"city": "munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_qKDV1MhGXWoGQtTstCvm8Inr', 'function': {'arguments': '{"city": "munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 98, 'total_tokens': 148}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1e34b739-184a-443a-83c9-71f06aadc292-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'munich'}, 'id': 'call_qcyCL7l0m8TSeAViDuLe4Ci2', 'type': 'tool_call'}, {'name': 'outdoor_seating_availability', 'args': {'city': 'munich'}, 'id': 'call_qKDV1MhGXWoGQtTstCvm8Inr', 'type': 'tool_call'}], usage_metadata={'input_tokens': 98, 'output_tokens': 50, 'total_tokens': 148})

In [8]:
result.tool_calls

[{'name': 'fake_weather_api',
  'args': {'city': 'munich'},
  'id': 'call_qcyCL7l0m8TSeAViDuLe4Ci2',
  'type': 'tool_call'},
 {'name': 'outdoor_seating_availability',
  'args': {'city': 'munich'},
  'id': 'call_qKDV1MhGXWoGQtTstCvm8Inr',
  'type': 'tool_call'}]

In [9]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage("How will the weather be in munich today? I would like to eat outside if possible")]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [10]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nOfS1hw7thd17nGayeJggBZX', 'function': {'arguments': '{"city": "munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_2yLvf8amDVKmmy0FJeEZXihL', 'function': {'arguments': '{"city": "munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 98, 'total_tokens': 148}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c2356012-ce73-4d0e-92a2-2a264fa562dd-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'munich'}, 'id': 'call_nOfS1hw7thd17nGayeJggBZX', 'type': 'tool_call'}, {'name': 'outdoor_seating_availability', 'args': {'city': 'munich'}, 'id': 'call_2yLvf8amDVKmmy0FJeEZXihL', 'type': 'tool_call'}], usage_metadata={'in

In [11]:
tool_mapping = {"fake_weather_api": fake_weather_api, "outdoor_seating_availability": outdoor_seating_availability}

In [12]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [13]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nOfS1hw7thd17nGayeJggBZX', 'function': {'arguments': '{"city": "munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_2yLvf8amDVKmmy0FJeEZXihL', 'function': {'arguments': '{"city": "munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 98, 'total_tokens': 148}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c2356012-ce73-4d0e-92a2-2a264fa562dd-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'munich'}, 'id': 'call_nOfS1hw7thd17nGayeJggBZX', 'type': 'tool_call'}, {'name': 'outdoor_seating_availability', 'args': {'city': 'munich'}, 'id': 'call_2yLvf8amDVKmmy0FJeEZXihL', 'type': 'tool_call'}], usage_metadata={'in

In [14]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Munich today is sunny with a temperature of 22°C. Outdoor seating is available, so you can enjoy your meal outside.', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 176, 'total_tokens': 205}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9bd81dfc-5b4e-48aa-9e51-83e8325f51d0-0', usage_metadata={'input_tokens': 176, 'output_tokens': 29, 'total_tokens': 205})

### Real third party API Call - 이건 스킾

In [15]:
from langchain_core.tools import tool
import httpx

@tool
def fake_weather_api(city: str) -> str:
    """Check the weather in a specified city from a FastAPI endpoint on localhost:8000."""
    # Make an HTTP GET request to the FastAPI endpoint
    response = httpx.get(f"http://localhost:8000/weather?city={city}")

    if response.status_code == 200:
        return response.json().get("weather", "Weather information not available")
    else:
        return "Failed to get weather information"


@tool
def outdoor_seating_availability(city: str) -> str:
    """Check if outdoor seating is available in a specified city from a FastAPI endpoint on localhost:8000."""
    # Make an HTTP GET request to the FastAPI endpoint
    response = httpx.get(f"http://localhost:8000/outdoor-seating?city={city}")

    if response.status_code == 200:
        return response.json().get("outdoor_seating", "Outdoor seating information not available")
    else:
        return "Failed to get outdoor seating information"


api_tools = [fake_weather_api, outdoor_seating_availability]
tool_mapping = {"fake_weather_api": fake_weather_api, "outdoor_seating_availability": outdoor_seating_availability}

In [16]:
from langchain_openai import ChatOpenAI

def interact_with_llm_and_tools(human_message: str):

    llm = ChatOpenAI()
    llm_with_tools_new = llm.bind_tools(api_tools)

    messages = [human_message]

    llm_output = llm_with_tools_new.invoke(messages)
    messages.append(llm_output)

    for tool_call in llm_output.tool_calls:
        tool_name = tool_call["name"].lower()
        tool = tool_mapping.get(tool_name)

        if tool:
            tool_output = tool.invoke(tool_call["args"])
            messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

    final_response = llm_with_tools_new.invoke(messages)
    return final_response

In [17]:
interact_with_llm_and_tools("How will the weather be in sunnytown today? I would like to eat outside if possible")

ConnectError: [Errno 99] Cannot assign requested address

In [18]:
interact_with_llm_and_tools("How will the weather be in rainytown today? I would like to eat outside if possible")

ConnectError: [Errno 99] Cannot assign requested address

In [19]:
interact_with_llm_and_tools("How will the weather be in munich today? I would like to eat outside if possible")

ConnectError: [Errno 99] Cannot assign requested address